In [1]:
from distributed import Client, LocalCluster
from dask import delayed
import dask

In [2]:
client1 = Client()  # starts a Localcluster automatically
client2 = Client()

/home/balast/.conda/envs/dask/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35439 instead
  warnings.warn(


In [3]:
def execute_sim(arg1, arg2, arg3):
    return arg1+arg2+arg3

simulation_arg1s = [1, 2, 3, 4]
simulation_arg2s = [5, 6, 7, 8]
simulation_arg3s = [9, 8, 7, 6]

## Futures Example

In [4]:
futures = client1.map(execute_sim, simulation_arg1s, simulation_arg2s, simulation_arg3s)  # begin computation now

results = client1.gather(futures)  # stop execution of this thread until futures are complete
results

[15, 16, 17, 18]

## Delayed Example

In [5]:
delayed_execute_sim = delayed(execute_sim)
delayed_execute_sim

Delayed('execute_sim-3ad26214-db1b-4b57-8809-0010231f81fd')

In [6]:
delayed_result = delayed_execute_sim(1,2,3)

# get the result 
result = dask.compute(delayed_result) # begins computation, returns a tuple
print(f'result is {result}')

result is (6,)


In [7]:
delayed_results = []
for arg1, arg2, arg3 in zip(simulation_arg1s, simulation_arg2s, simulation_arg3s):
    result = delayed_execute_sim(arg1, arg2, arg3)
    delayed_results.append(result)

dask.compute(*delayed_results)  # begins computation

(15, 16, 17, 18)

## Run on cluster 1

In [11]:
futures = client1.compute(delayed_results)
results = client1.gather(futures)
results

[15, 16, 17, 18]

## Run on cluster 2

In [12]:
futures = client2.compute(delayed_results)
results = client2.gather(futures)
results

[15, 16, 17, 18]

In [18]:
# can see the computation has occurred on both clusters by looking at the dashboards
client1.dashboard_link, client2.dashboard_link

('http://127.0.0.1:8787/status', 'http://127.0.0.1:35439/status')